In [0]:
# Imports
import json
from typing import Annotated, Any, Generator, Optional, Sequence, TypedDict, Union
from uuid import uuid4

import mlflow
from databricks_langchain import (
    ChatDatabricks,
    UCFunctionToolkit,
    VectorSearchRetrieverTool,
)
from langchain_core.language_models import LanguageModelLike
from langchain_core.messages import (
    AIMessage,
    AIMessageChunk,
    BaseMessage,
    convert_to_openai_messages,
)
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.tools import BaseTool
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_node import ToolNode
from mlflow.entities import SpanType
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
)

from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)

from langgraph.graph.state import CompiledStateGraph


In [0]:

LLM_ENDPOINT_NAME = "databricks-gpt-oss-120b"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)


system_prompt = """You are the Foodly Escalation Agent.
Your job is to escalate a conversation to a human support specialist.

You MUST always know the user’s ID (user_id) or email_id before creating a ticket.

If user_id is not provided in the conversation context, do not guess or proceed.
Instead, ask the user politely for their registered email or phone number, or any ID you can use to look up their account.

Once you have user_id, use the escalation tool to create a support ticket.
Confirm to the user that their case has been escalated, and provide the ticket ID and ETA from the tool output.

"""

###############################################################################
## Define tools for your agent, enabling it to retrieve data or take actions
## beyond text generation
## To create and see usage examples of more tools, see
## https://docs.databricks.com/en/generative-ai/agent-framework/agent-tool.html
###############################################################################

uc_tool_names  = ("agents.escalation.*",)


tools  = UCFunctionToolkit(function_names=list(uc_tool_names)).tools



from helpers import LangGraphResponsesAgent, create_tool_calling_agent
mlflow.langchain.autolog()
agent = create_tool_calling_agent(llm, tools, system_prompt)
AGENT = LangGraphResponsesAgent(agent)
mlflow.models.set_model(AGENT)





In [0]:
for chunk in AGENT.predict_stream(
    {"input": [{"role": "user", "content": "I want to speak to the manager"},
               {"role": "user", "content": "my email id is anirvan.sen@databricks.com"}]}
):
    print(chunk.model_dump(exclude_none=True))

### Creating tools for escalation agent , this will be uc functions

In [0]:
%sql
create schema if not exists agents.escalation;

In [0]:
%sql
CREATE OR REPLACE FUNCTION agents.escalation.escalate_to_human(
    user_id_input STRING COMMENT 'User ID needing escalation',
    summary_input STRING COMMENT 'Brief summary of the conversation or issue'
)
RETURNS TABLE (
    ticket_id STRING,
    eta_minutes INT,
    message STRING
)
COMMENT 'Creates a support ticket for human intervention, returns a ticket ID, estimated response time, and a user-facing message.'
RETURN (
    SELECT
      CONCAT('TCK-', CAST(FLOOR(RAND() * 1000000) AS STRING)) AS ticket_id,
      30 AS eta_minutes,
      CONCAT('We have escalated your issue to a human support specialist. You can expect a response within ', 30, ' minutes.') AS message
);
